<a href="https://colab.research.google.com/github/souradip93/CS69002_9A_18CS60R07/blob/master/Assignment9A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import the libraries**

In [0]:
import torch
import pandas as pd
import numpy as np
import io
import nltk
import re

# **Upload Train file**

Option 1 - Upload the trainig dataset

In [0]:
from google.colab import files
uploaded = files.upload()
df_train = pd.read_csv(io.StringIO(uploaded['Train_20K.txt'].decode('utf-8')), sep='\t')
df_train.head()


uploaded = files.upload()
df_test = pd.read_csv(io.StringIO(uploaded['Test_5K.txt'].decode('utf-8')), sep='\t')
df_test.head()

*OR*

Option 2 - Store the file in your google drive account

In [2]:
from google.colab import drive
drive.mount('/content/drive')
df_train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/Train_20K.txt', sep='\t')
df_train.head()
df_test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/Test_5K.txt', sep='\t')
df_test.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,text,label
0,as always this is an inaccurate picture of the...,0
1,Did the movie-makers even preview this before ...,0
2,"Heavily re-edited and often confusing, the ori...",0
3,I notice that most of the people who think thi...,0
4,"First of all, this is a low-budget movie, so m...",0


In [0]:
train_data_X = df_train['text'].astype(str).tolist()
train_data_Y = df_train['label'].astype(int)

In [0]:
test_data_X = df_test['text'].astype(str).tolist()
test_data_Y = df_test['label'].astype(int)

# **Preprocessing**

1. Remove br tags and numbers

In [0]:
def remove_br_tags_and_numbers(train_data_X):
  import re
  regex = re.compile(r'<.*?>|\d+')
  train_data_X = [regex.sub(' ', x) for x in train_data_X]
  train_data_X = [x.replace("'", '').replace('\n\n', ' ') for x in train_data_X]
  return train_data_X

2. Tokenize - Consider only alphanumeric characters

In [0]:

def tokenize(train_data_X):
  from nltk.tokenize import RegexpTokenizer
  tokenizer = RegexpTokenizer(r'\w+')
  train_data_X = [tokenizer.tokenize(x) for x in train_data_X]
  #train_data_X = [x.split(' ') for x in train_data_X]
  #train_data_X = [[y for y in x if y != ''] for x in train_data_X]
  #train_data_X[0], train_data_Y[0]
  return train_data_X


3. Convert to lower case

In [0]:
def convert_to_lower(train_data_X):
  train_data_X = [[y.lower() for y in x] for x in train_data_X]
  #train_data_X[0], train_data_Y[0]
  return train_data_X

4. Remove stopwords

In [0]:
def remove_stopwords(train_data_X):
  from nltk.corpus import stopwords
  nltk.download('stopwords')
  stopword_set = set(stopwords.words('english'))
  train_data_X = [[y for y in x if y not in stopword_set] for x in train_data_X]
  #train_data_X[0], train_data_Y[0]
  return train_data_X

5. Perform lemmatization

In [0]:
def lemmatize(train_data_X):
  from nltk.stem import WordNetLemmatizer 
  nltk.download('wordnet')
  lemmatizer = WordNetLemmatizer()
  train_data_X = [[lemmatizer.lemmatize(lemmatizer.lemmatize(lemmatizer.lemmatize(lemmatizer.lemmatize(y,'n'), 'v'), 'a'), 'r') for y in x] for x in train_data_X]
  return train_data_X

6. Remove punctuation

In [0]:
def remove_punctuation(train_data_X):
  import string
  table = str.maketrans(' ', ' ', string.punctuation)
  train_data_X = [x.translate(table) for x in train_data_X]
  return train_data_X

**Preprocessing on Training data**

In [11]:
train_data_X = remove_br_tags_and_numbers(train_data_X)
#train_data_X = remove_punctuation(train_data_X)
train_data_X = tokenize(train_data_X)
train_data_X = convert_to_lower(train_data_X)
train_data_X = remove_stopwords(train_data_X)
train_data_X = lemmatize(train_data_X)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


**Preprocessing on Test data**

In [12]:
test_data_X = remove_br_tags_and_numbers(test_data_X)
#test_data_X = remove_punctuation(test_data_X)
test_data_X = tokenize(test_data_X)
test_data_X = convert_to_lower(test_data_X)
test_data_X = remove_stopwords(test_data_X)
test_data_X = lemmatize(test_data_X)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#**Reduce Vocab**

In [0]:
def generate_word_frequency(dataset):
  word_frequency = {}
  for sent in dataset:
      for word in sent:
          if word not in word_frequency:
              word_frequency[word] = 1
          else:
              val = word_frequency[word]
              val += 1
              word_frequency[word] = val
              
  #word_to_ix['UNKNOWN'] = len(word_to_ix)
  return word_frequency

In [0]:
THRESHOLD_FREQUENCY = 10

def reduce_vocab(word_frequency):
  word_to_ix = {}
  for word in word_frequency:
    val = word_frequency[word]
    if val >= THRESHOLD_FREQUENCY:
      word_to_ix[word] = len(word_to_ix)
  word_to_ix['UNKNOWN'] = len(word_to_ix)
  return word_to_ix

In [15]:
word_frequency = generate_word_frequency(train_data_X + test_data_X)
word_to_ix = reduce_vocab(word_frequency)

VOCAB_SIZE = len(word_to_ix)
print(VOCAB_SIZE)

14764


Store the word index once created

In [0]:
import pickle

pickle_out = open("word_to_ix.pickle","wb")
pickle.dump(word_to_ix, pickle_out)
pickle_out.close()

from google.colab import files
model = files.download("word_to_ix.pickle")


#**Reduce Vocab - Bigram**

In [0]:
def generate_word_frequency_bigram(dataset):
  word_frequency = {}
  for sent in dataset:
      for ind, word1 in enumerate(sent[:-1]):
          word2 = sent[ind+1]
          if (word1,word2) not in word_frequency :
              word_frequency[(word1,word2)] = 1
          else:
              val = word_frequency[(word1,word2)]
              val += 1
              word_frequency[(word1,word2)] = val
              
  #word_to_ix['UNKNOWN'] = len(word_to_ix)
  return word_frequency

In [0]:
THRESHOLD_FREQUENCY = 5

def reduce_vocab_bigram(word_frequency):
  word_to_ix = {}
  for (word1,word2) in word_frequency:
    val = word_frequency[(word1,word2)]
    if val >= THRESHOLD_FREQUENCY:
      word_to_ix[(word1,word2)] = len(word_to_ix)
  word_to_ix['UNKNOWN'] = len(word_to_ix)
  return word_to_ix

In [132]:
word_frequency_bigram = generate_word_frequency_bigram(train_data_X + test_data_X)
word_to_ix_bigram = reduce_vocab_bigram(word_frequency_bigram)

VOCAB_SIZE = len(word_to_ix_bigram)
print(VOCAB_SIZE)

71625


Store the word index once created

In [0]:
import pickle

pickle_out = open("word_to_ix_bigram.pickle","wb")
pickle.dump(word_to_ix_bigram, pickle_out)
pickle_out.close()

from google.colab import files
model = files.download("word_to_ix_bigram.pickle")


Irrelevant for this Task. Remove it later

In [19]:
label_to_ix = {1: 1, 0: 0}
ix_to_label = {v: k for k, v in label_to_ix.items()}

label_to_ix, ix_to_label

({0: 0, 1: 1}, {0: 0, 1: 1})

#**Pytorch initialization**

In [17]:
import torch.nn as nn
import torch.nn.functional as F

SEED = 42

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.device('cuda')



device(type='cuda')

#**Create bow vectors**

In [0]:
from torch.autograd import Variable
import time

def make_bow_vector(sentence, word_to_ix):
    # create a vector of zeros of vocab size = len(word_to_idx)
    vec = torch.zeros(len(word_to_ix))
    for word in sentence:
        if word not in word_to_ix:
            #raise ValueError('Word',word,' not present in the dictionary. Sorry!')
            vec[ word_to_ix['UNKNOWN'] ] += 1
        else:
            vec[word_to_ix[word]]+=1
    return vec.view(1, -1)

def make_target(label, label_to_ix):
    #print(label)
    return torch.LongTensor([label_to_ix[label]])

bow_vec_orig = []
for instance in train_data_X:
  ins_bow_vector = make_bow_vector(instance, word_to_ix)
  bow_vec_orig.append(Variable(ins_bow_vector).cuda())
bow_vec_orig = torch.stack(bow_vec_orig).cuda()
  
label_orig = []
for l in train_data_Y:
  label_vector = make_target(l, label_to_ix)
  label_orig.append(Variable(label_vector).cuda())
label_orig = torch.stack(label_orig).cuda()

#**Create bow vectors - Bigram**

In [0]:
from torch.autograd import Variable
import time

def make_bow_vector_bigram(sentence, word_to_ix_bigram):
    # create a vector of zeros of vocab size = len(word_to_idx)
    vec = torch.zeros(len(word_to_ix_bigram))
    for ind,word1 in enumerate(sentence[:-1]):
        word2 = sentence[ind+1]
        if (word1, word2) not in word_to_ix_bigram:
            #raise ValueError('Word',word,' not present in the dictionary. Sorry!')
            #vec[ word_to_ix_bigram['UNKNOWN'] ] += 1
            pass
        else:
            vec[word_to_ix_bigram[(word1, word2)]]+=1
    return vec.view(1, -1)

def make_target(label, label_to_ix):
    #print(label)
    return torch.LongTensor([label_to_ix[label]])

bow_vec_bgram_orig = []
for instance in train_data_X:
  ins_bow_vector = make_bow_vector_bigram(instance, word_to_ix_bigram)
  bow_vec_bgram_orig.append(Variable(ins_bow_vector).cuda())
bow_vec_bgram_orig = torch.stack(bow_vec_bgram_orig).cuda()

#**Functions to train model and find metrics**

In [0]:
import torch
from google.colab import files
  
def train(task, loss_function, opt):
  bow_vec = bow_vec_orig[:]
  label = label_orig[:]
  # the training loop
  for epoch in range(10):
      start = time.time()
      for ind, instance in enumerate(train_data_X):
          task.zero_grad()
          probs = task(bow_vec[ind]) # forward pass
          loss = loss_function(probs, label[ind])
          loss.backward()
          opt.step()
      end = time.time()
      print('Time Taken - ' + str(end - start))
      print('Epoch - ' + str(epoch) + ' , LOSS - ' + str(loss.data))
  return task
      
def calculateMetrics(task):
  print('--- AFTER TRAINING ---')
  tp = 0
  tn = 0
  fp = 0
  fn = 0

  for ind, instance in enumerate(test_data_X):
      bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
      logprobs = task(bow_vec)
      pred = np.argmax(logprobs.cpu().data.numpy())
      label = test_data_Y[ind]
      if label == 1 and ix_to_label[pred] == 1:
        tp += 1
      elif label == 1 and ix_to_label[pred] == 0:
        fn += 1
      elif label == 0 and ix_to_label[pred] == 0:
        tn += 1
      else:
        fp += 1

  accuracy = float(tp+tn)/ len(test_data_X)
  recall = float(tp)/ (tp+fn)
  precision = float(tp)/ (tp+fp)
  fmeasure = float(2*recall*precision) / (precision+recall)
  
  print('Accuracy - ' + str(accuracy))
  print('Recall - ' + str(recall))
  print('Precision - ' + str(precision))
  print('Fmeasure - ' + str(fmeasure))
  
def save_model(task, name):
  torch.save(task,name)
  files.download(name)

#**Neural network for Task 1**

In [0]:
class Task1(nn.Module):
  def __init__(self, num_labels, hidden_layer_neurons, vocab_size):
    super(Task1, self).__init__()
    self.lin1 = nn.Linear(vocab_size, hidden_layer_neurons)
    self.lin2 = nn.Linear(hidden_layer_neurons, num_labels)
    
  def forward(self, x):
    out = self.lin1(x)
    return F.softmax(self.lin2(out))

Setting model parameters for Task 1 A

In [0]:
NUM_LABELS = 2  
HIDDEN_LAYER_NEURONS = 50
  
task1a = Task1(NUM_LABELS, HIDDEN_LAYER_NEURONS, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function1a = nn.NLLLoss()
opt1a = torch.optim.SGD(task1a.parameters(), lr = 0.01)

Setting model parameters for Task 1 B

In [0]:
NUM_LABELS = 2  
HIDDEN_LAYER_NEURONS = 100
  
task1b = Task1(NUM_LABELS, HIDDEN_LAYER_NEURONS, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function1b = nn.NLLLoss()
opt1b = torch.optim.SGD(task1b.parameters(), lr = 0.01)

Setting model parameters for Task 1 C

In [0]:
NUM_LABELS = 2  
HIDDEN_LAYER_NEURONS = 150
  
task1c = Task1(NUM_LABELS, HIDDEN_LAYER_NEURONS, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function1c = nn.NLLLoss()
opt1c = torch.optim.SGD(task1c.parameters(), lr = 0.01)

#**Training for Task 1**

**Task 1 a**

*TRAINING*

In [60]:
task = train(task1a, loss_function1a, opt1a)
calculateMetrics(task)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':


Time Taken - 16.32853102684021
Epoch - 0 , LOSS - tensor(-1.0000, device='cuda:0')
Time Taken - 16.219228982925415
Epoch - 1 , LOSS - tensor(-1.0000, device='cuda:0')
Time Taken - 16.209766626358032
Epoch - 2 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 16.11113142967224
Epoch - 3 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 16.14791774749756
Epoch - 4 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 16.643018007278442
Epoch - 5 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 16.039944171905518
Epoch - 6 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 16.057997465133667
Epoch - 7 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 16.17435932159424
Epoch - 8 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 16.094764709472656
Epoch - 9 , LOSS - tensor(-1., device='cuda:0')
--- AFTER TRAINING ---
Accuracy - 0.868
Recall - 0.8705321683876092
Precision - 0.8677751385589866
Fmeasure - 0.8691514670896114


In [62]:
save_model(task.cpu(), 'model_1a.mdl')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Task1. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


**Task 1 b**

*TRAINING*

In [64]:
task = train(task1b, loss_function1b, opt1b)
calculateMetrics(task)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':


Time Taken - 16.44837236404419
Epoch - 0 , LOSS - tensor(-1.0000, device='cuda:0')
Time Taken - 16.170080184936523
Epoch - 1 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 16.250499725341797
Epoch - 2 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 16.150915145874023
Epoch - 3 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 16.178941249847412
Epoch - 4 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 17.09151005744934
Epoch - 5 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 16.30715847015381
Epoch - 6 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 16.360268354415894
Epoch - 7 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 16.28510856628418
Epoch - 8 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 16.321537971496582
Epoch - 9 , LOSS - tensor(-1., device='cuda:0')
--- AFTER TRAINING ---
Accuracy - 0.8698
Recall - 0.891977760127085
Precision - 0.8556190476190476
Fmeasure - 0.8734201827727007


In [65]:
save_model(task.cpu(), 'model_1b.mdl')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Task1. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


**Task 1 c**

*TRAINING*

In [67]:
task = train(task1c, loss_function1c, opt1c)
calculateMetrics(task)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':


Time Taken - 16.621872901916504
Epoch - 0 , LOSS - tensor(-1.0000, device='cuda:0')
Time Taken - 16.347761631011963
Epoch - 1 , LOSS - tensor(-1.0000, device='cuda:0')
Time Taken - 16.811615228652954
Epoch - 2 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 16.191262006759644
Epoch - 3 , LOSS - tensor(-1.0000, device='cuda:0')
Time Taken - 16.209725856781006
Epoch - 4 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 16.18377161026001
Epoch - 5 , LOSS - tensor(-1.0000, device='cuda:0')
Time Taken - 16.166773080825806
Epoch - 6 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 16.199344158172607
Epoch - 7 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 16.161771774291992
Epoch - 8 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 16.147169589996338
Epoch - 9 , LOSS - tensor(-1., device='cuda:0')
--- AFTER TRAINING ---
Accuracy - 0.8666
Recall - 0.8613979348689436
Precision - 0.8721351025331725
Fmeasure - 0.8667332667332667


In [68]:
save_model(task.cpu(), 'model_1c.mdl')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Task1. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


#**Neural network for Task 2**

In [0]:
class Task2(nn.Module):
  def __init__(self, num_labels, hidden_layer_neurons1, hidden_layer_neurons2, vocab_size):
    super(Task2, self).__init__()
    self.lin1 = nn.Linear(vocab_size, hidden_layer_neurons1)
    self.lin2 = nn.Linear(hidden_layer_neurons1, hidden_layer_neurons2)
    self.lin3 = nn.Linear(hidden_layer_neurons2, num_labels)
    
  def forward(self, x):
    out = self.lin1(x)
    out = F.relu(self.lin2(out))
    return F.softmax(self.lin3(out))
    

Setting model parameters for Task 2 A

In [0]:
NUM_LABELS = 2  
HIDDEN_LAYER_NEURONS1 = 10
HIDDEN_LAYER_NEURONS2 = 10
  
task2a = Task2(NUM_LABELS, HIDDEN_LAYER_NEURONS1, HIDDEN_LAYER_NEURONS2, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function2a = nn.NLLLoss()
opt2a = torch.optim.SGD(task2a.parameters(), lr = 0.01)

Setting model parameters for Task 2 B

In [0]:
NUM_LABELS = 2  
HIDDEN_LAYER_NEURONS1 = 20
HIDDEN_LAYER_NEURONS2 = 10
  
task2b = Task2(NUM_LABELS, HIDDEN_LAYER_NEURONS1, HIDDEN_LAYER_NEURONS2, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function2b = nn.NLLLoss()
opt2b = torch.optim.SGD(task2b.parameters(), lr = 0.01)

Setting model parameters for Task 2 C

In [0]:
NUM_LABELS = 2  
HIDDEN_LAYER_NEURONS1 = 30
HIDDEN_LAYER_NEURONS2 = 30
  
task2c = Task2(NUM_LABELS, HIDDEN_LAYER_NEURONS1, HIDDEN_LAYER_NEURONS2, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function2c = nn.NLLLoss()
opt2c = torch.optim.SGD(task2c.parameters(), lr = 0.01)

Setting model parameters for Task 2 D

In [0]:
NUM_LABELS = 2  
HIDDEN_LAYER_NEURONS1 = 50
HIDDEN_LAYER_NEURONS2 = 50
  
task2d = Task2(NUM_LABELS, HIDDEN_LAYER_NEURONS1, HIDDEN_LAYER_NEURONS2, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function2d = nn.NLLLoss()
opt2d = torch.optim.SGD(task2d.parameters(), lr = 0.01)

Setting model parameters for Task 2 E

In [0]:
NUM_LABELS = 2  
HIDDEN_LAYER_NEURONS1 = 100
HIDDEN_LAYER_NEURONS2 = 50
  
task2e = Task2(NUM_LABELS, HIDDEN_LAYER_NEURONS1, HIDDEN_LAYER_NEURONS2, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function2e = nn.NLLLoss()
opt2e = torch.optim.SGD(task2e.parameters(), lr = 0.01)

#**Training for Task 2**

**Task 2 a**

*TRAINING*

In [71]:
task = train(task2a, loss_function2a, opt2a)
calculateMetrics(task)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # This is added back by InteractiveShellApp.init_path()


Time Taken - 20.332382678985596
Epoch - 0 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.249682903289795
Epoch - 1 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.878215789794922
Epoch - 2 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.553465843200684
Epoch - 3 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.411340475082397
Epoch - 4 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.410038471221924
Epoch - 5 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.605851888656616
Epoch - 6 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.357484579086304
Epoch - 7 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.44930624961853
Epoch - 8 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.380980491638184
Epoch - 9 , LOSS - tensor(-1., device='cuda:0')
--- AFTER TRAINING ---
Accuracy - 0.8486
Recall - 0.903494837172359
Precision - 0.8157045536034421
Fmeasure - 0.8573582061428301


In [72]:
save_model(task.cpu(), 'model_2a.mdl')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Task2. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


**Task 2 b**

*TRAINING*

In [91]:
task = train(task2b, loss_function2b, opt2b)
calculateMetrics(task)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # This is added back by InteractiveShellApp.init_path()


Time Taken - 20.613932371139526
Epoch - 0 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.29813027381897
Epoch - 1 , LOSS - tensor(-1.0000, device='cuda:0')
Time Taken - 20.358999490737915
Epoch - 2 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.37652039527893
Epoch - 3 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.391200304031372
Epoch - 4 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.37814736366272
Epoch - 5 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.308969020843506
Epoch - 6 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.433732509613037
Epoch - 7 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.464009046554565
Epoch - 8 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.343831777572632
Epoch - 9 , LOSS - tensor(-1., device='cuda:0')
--- AFTER TRAINING ---
Accuracy - 0.8436
Recall - 0.8018268467037332
Precision - 0.8770634231103388
Fmeasure - 0.8377593360995851


In [92]:
save_model(task.cpu(), 'model_2b.mdl')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Task2. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


**Task 2c**

*TRAINING*

In [93]:
task = train(task2c, loss_function2c, opt2c)
calculateMetrics(task)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # This is added back by InteractiveShellApp.init_path()


Time Taken - 20.927616119384766
Epoch - 0 , LOSS - tensor(-1.0000, device='cuda:0')
Time Taken - 20.341686964035034
Epoch - 1 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.189078330993652
Epoch - 2 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.60692286491394
Epoch - 3 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 19.986009120941162
Epoch - 4 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.11312508583069
Epoch - 5 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.151569843292236
Epoch - 6 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.112948656082153
Epoch - 7 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.098836660385132
Epoch - 8 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.17791199684143
Epoch - 9 , LOSS - tensor(-1., device='cuda:0')
--- AFTER TRAINING ---
Accuracy - 0.8588
Recall - 0.8629864972200159
Precision - 0.8575374901341752
Fmeasure - 0.8602533650039589


In [94]:
save_model(task.cpu(), 'model_2c.mdl')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Task2. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


**Task 2 d**

*TRAINING*

In [95]:
task = train(task2d, loss_function2d, opt2d)
calculateMetrics(task)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # This is added back by InteractiveShellApp.init_path()


Time Taken - 20.10313367843628
Epoch - 0 , LOSS - tensor(-1.0000, device='cuda:0')
Time Taken - 20.005864143371582
Epoch - 1 , LOSS - tensor(-1.0000, device='cuda:0')
Time Taken - 19.86429762840271
Epoch - 2 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 19.96662211418152
Epoch - 3 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.580289363861084
Epoch - 4 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.09446930885315
Epoch - 5 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.0238299369812
Epoch - 6 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.43057107925415
Epoch - 7 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.105145692825317
Epoch - 8 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.001198053359985
Epoch - 9 , LOSS - tensor(-1., device='cuda:0')
--- AFTER TRAINING ---
Accuracy - 0.858
Recall - 0.892374900714853
Precision - 0.836559940431869
Fmeasure - 0.8635664873174481


In [96]:
save_model(task.cpu(), 'model_2d.mdl')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Task2. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


**Task 2 e**

*TRAINING*

In [97]:
task = train(task2e, loss_function2e, opt2e)
calculateMetrics(task)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # This is added back by InteractiveShellApp.init_path()


Time Taken - 20.08453941345215
Epoch - 0 , LOSS - tensor(-1.0000, device='cuda:0')
Time Taken - 20.026304483413696
Epoch - 1 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 19.976668119430542
Epoch - 2 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 19.97039771080017
Epoch - 3 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.02477788925171
Epoch - 4 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 19.99086332321167
Epoch - 5 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.005309104919434
Epoch - 6 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.02510380744934
Epoch - 7 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 19.977061986923218
Epoch - 8 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.671231746673584
Epoch - 9 , LOSS - tensor(-1., device='cuda:0')
--- AFTER TRAINING ---
Accuracy - 0.8546
Recall - 0.8776806989674345
Precision - 0.8406238113351084
Fmeasure - 0.8587526714591024


In [98]:
save_model(task.cpu(), 'model_2e.mdl')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Task2. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


#**Neural network for Task 3**

In [0]:
class Task3(nn.Module):
  def __init__(self, num_labels, hidden_layer_neurons1, hidden_layer_neurons2, hidden_layer_neurons3, vocab_size):
    super(Task3, self).__init__()
    self.lin1 = nn.Linear(vocab_size, hidden_layer_neurons1)
    self.lin2 = nn.Linear(hidden_layer_neurons1, hidden_layer_neurons2)
    self.lin3 = nn.Linear(hidden_layer_neurons2, hidden_layer_neurons3)
    self.lin4 = nn.Linear(hidden_layer_neurons3, num_labels)
    
  def forward(self, x):
    out = self.lin1(x)
    out = F.relu(self.lin2(out))
    out = self.lin3(out)
    return F.softmax(self.lin4(out))

Setting model parameters for Task 3 A

In [0]:
NUM_LABELS = 2  
HIDDEN_LAYER_NEURONS1 = 100
HIDDEN_LAYER_NEURONS2 = 50
HIDDEN_LAYER_NEURONS3 = 10
  
task3a = Task3(NUM_LABELS, HIDDEN_LAYER_NEURONS1, HIDDEN_LAYER_NEURONS2, HIDDEN_LAYER_NEURONS3, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function3a = nn.NLLLoss()
opt3a = torch.optim.SGD(task3a.parameters(), lr = 0.01)

Setting model parameters for Task 3 B

In [0]:
NUM_LABELS = 2  
HIDDEN_LAYER_NEURONS1 = 200
HIDDEN_LAYER_NEURONS2 = 100
HIDDEN_LAYER_NEURONS3 = 10
  
task3b = Task3(NUM_LABELS, HIDDEN_LAYER_NEURONS1, HIDDEN_LAYER_NEURONS2, HIDDEN_LAYER_NEURONS3, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function3b = nn.NLLLoss()
opt3b = torch.optim.SGD(task3b.parameters(), lr = 0.01)

#**Training for Task 3**

**Task 3 a**

*TRAINING*

In [108]:
task = train(task3a, loss_function3a, opt3a)
calculateMetrics(task)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  del sys.path[0]


Time Taken - 24.067967891693115
Epoch - 0 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 24.127845287322998
Epoch - 1 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 23.986689567565918
Epoch - 2 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 24.021493673324585
Epoch - 3 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 24.00989270210266
Epoch - 4 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 24.25253391265869
Epoch - 5 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 24.2269868850708
Epoch - 6 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 24.105911254882812
Epoch - 7 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 24.07929039001465
Epoch - 8 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 23.9444317817688
Epoch - 9 , LOSS - tensor(-1., device='cuda:0')
--- AFTER TRAINING ---
Accuracy - 0.8398
Recall - 0.9007148530579825
Precision - 0.8045406172401561
Fmeasure - 0.8499156829679595


In [109]:
save_model(task.cpu(), 'model_3a.mdl')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Task3. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


**Task 3 b**

*TRAINING*

In [110]:
task = train(task3b, loss_function3b, opt3b)
calculateMetrics(task)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  del sys.path[0]


Time Taken - 24.582305908203125
Epoch - 0 , LOSS - tensor(-1.0000, device='cuda:0')
Time Taken - 24.514402151107788
Epoch - 1 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 24.35111165046692
Epoch - 2 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 24.364917516708374
Epoch - 3 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 24.37273335456848
Epoch - 4 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 24.252920150756836
Epoch - 5 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 24.342594861984253
Epoch - 6 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 24.94948935508728
Epoch - 7 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 24.57725191116333
Epoch - 8 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 24.809766054153442
Epoch - 9 , LOSS - tensor(-1., device='cuda:0')
--- AFTER TRAINING ---
Accuracy - 0.8126
Recall - 0.6894360603653693
Precision - 0.9180327868852459
Fmeasure - 0.7874801542299841


In [111]:
save_model(task.cpu(), 'model_3b.mdl')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Task3. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


#**Neural network for Task 4**

In [0]:
class Task4(nn.Module):
  def __init__(self, num_labels, hidden_layer_neurons1, hidden_layer_neurons2, hidden_layer_neurons3, vocab_size):
    super(Task4, self).__init__()
    self.lin1 = nn.Linear(vocab_size, hidden_layer_neurons1)
    torch.nn.Dropout(0.5)
    self.lin2 = nn.Linear(hidden_layer_neurons1, hidden_layer_neurons2)
    torch.nn.Dropout(0.5)
    if hidden_layer_neurons3 is not None:
      self.lin3 = nn.Linear(hidden_layer_neurons2, hidden_layer_neurons3)
      torch.nn.Dropout(0.5)
      self.lin4 = nn.Linear(hidden_layer_neurons3, num_labels)
    else:
      self.lin3 = nn.Linear(hidden_layer_neurons2, num_labels)
      self.lin4 = None
    
  def forward(self, x):
    out = self.lin1(x)
    out = F.relu(self.lin2(out))
    out = self.lin3(out)
    if self.lin4 is None:
      return F.softmax(out)
    else:
      return F.softmax(self.lin4(out))

Setting model parameters for Task 4 A

In [0]:
NUM_LABELS = 2  
HIDDEN_LAYER_NEURONS1 = 30
HIDDEN_LAYER_NEURONS2 = 20
HIDDEN_LAYER_NEURONS3 = 10
  
task4a = Task4(NUM_LABELS, HIDDEN_LAYER_NEURONS1, HIDDEN_LAYER_NEURONS2, HIDDEN_LAYER_NEURONS3, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function4a = nn.NLLLoss()
opt4a = torch.optim.SGD(task4a.parameters(), lr = 0.01)

Setting model parameters for Task 4 B

In [0]:
NUM_LABELS = 2  
HIDDEN_LAYER_NEURONS1 = 100
HIDDEN_LAYER_NEURONS2 = 100
  
task4b = Task4(NUM_LABELS, HIDDEN_LAYER_NEURONS1, HIDDEN_LAYER_NEURONS2, None, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function4b = nn.NLLLoss()
opt4b = torch.optim.SGD(task4b.parameters(), lr = 0.01)

Setting model parameters for Task 4 C

In [0]:
NUM_LABELS = 2  
HIDDEN_LAYER_NEURONS1 = 100
HIDDEN_LAYER_NEURONS2 = 10
  
task4c = Task4(NUM_LABELS, HIDDEN_LAYER_NEURONS1, HIDDEN_LAYER_NEURONS2, None, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function4c = nn.NLLLoss()
opt4c = torch.optim.SGD(task4c.parameters(), lr = 0.01)

#**Training for Task 4**

**Task 4 a**

*TRAINING*

In [119]:
task = train(task4a, loss_function4a, opt4a)
calculateMetrics(task)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Time Taken - 24.308379888534546
Epoch - 0 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 24.271718502044678
Epoch - 1 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 24.03266429901123
Epoch - 2 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 24.091163873672485
Epoch - 3 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 24.035337209701538
Epoch - 4 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 24.033724546432495
Epoch - 5 , LOSS - tensor(-1.0000, device='cuda:0')
Time Taken - 24.09830665588379
Epoch - 6 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 24.136087656021118
Epoch - 7 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 24.061615705490112
Epoch - 8 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 24.016504049301147
Epoch - 9 , LOSS - tensor(-1., device='cuda:0')
--- AFTER TRAINING ---
Accuracy - 0.8554
Recall - 0.846306592533757
Precision - 0.8638021888933928
Fmeasure - 0.8549648946840522


In [120]:
save_model(task.cpu(), 'model_4a.mdl')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Task4. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


**Task 4  b**

*TRAINING*

In [124]:
task = train(task4b, loss_function4b, opt4b)
calculateMetrics(task)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Time Taken - 21.187002420425415
Epoch - 0 , LOSS - tensor(-1.0000, device='cuda:0')
Time Taken - 21.189964056015015
Epoch - 1 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.934136629104614
Epoch - 2 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.837417602539062
Epoch - 3 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.840358018875122
Epoch - 4 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.716098308563232
Epoch - 5 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.734814882278442
Epoch - 6 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.773621082305908
Epoch - 7 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.857794046401978
Epoch - 8 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.91875171661377
Epoch - 9 , LOSS - tensor(-1., device='cuda:0')
--- AFTER TRAINING ---
Accuracy - 0.8438
Recall - 0.8836378077839555
Precision - 0.8201253225211943
Fmeasure - 0.8506977633339706


In [125]:
save_model(task.cpu(), 'model_4b.mdl')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Task4. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


**Task 4  c**

*TRAINING*

In [128]:
task = train(task4c, loss_function4c, opt4c)
calculateMetrics(task)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Time Taken - 20.954166412353516
Epoch - 0 , LOSS - tensor(-1.0000, device='cuda:0')
Time Taken - 20.848313331604004
Epoch - 1 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.835992336273193
Epoch - 2 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.892149925231934
Epoch - 3 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.897969245910645
Epoch - 4 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.770830869674683
Epoch - 5 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.764296054840088
Epoch - 6 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 20.794527530670166
Epoch - 7 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 21.391910791397095
Epoch - 8 , LOSS - tensor(-1., device='cuda:0')
Time Taken - 21.16739511489868
Epoch - 9 , LOSS - tensor(-1., device='cuda:0')
--- AFTER TRAINING ---
Accuracy - 0.8506
Recall - 0.8907863383637807
Precision - 0.8261510128913444
Fmeasure - 0.8572520542709727


In [129]:
save_model(task.cpu(), 'model_4c.mdl')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Task4. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


#**Neural network for Task 5**

In [0]:
class Task5a(nn.Module):
  def __init__(self, num_labels, hidden_layer_neurons1, hidden_layer_neurons2, hidden_layer_neurons3, vocab_size):
    super(Task5a, self).__init__()
    self.lin1 = nn.Linear(vocab_size, hidden_layer_neurons1)
    torch.nn.Dropout(0.5)
    self.lin2 = nn.Linear(hidden_layer_neurons1, hidden_layer_neurons2)
    torch.nn.Dropout(0.5)
    if hidden_layer_neurons3 is not None:
      self.lin3 = nn.Linear(hidden_layer_neurons2, hidden_layer_neurons3)
      torch.nn.Dropout(0.5)
      self.lin4 = nn.Linear(hidden_layer_neurons3, num_labels)
    else:
      self.lin3 = nn.Linear(hidden_layer_neurons2, num_labels)
      self.lin4 = None
    
  def forward(self, x):
    out = self.lin1(x)
    out = F.relu(self.lin2(out))
    out = self.lin3(out)
    if self.lin4 is None:
      return F.softmax(out)
    else:
      return F.softmax(self.lin4(out))
    
class Task5b(nn.Module):
  def __init__(self, num_labels, hidden_layer_neurons1, hidden_layer_neurons2, hidden_layer_neurons3, vocab_size):
    super(Task5b, self).__init__()
    self.lin1 = nn.Linear(vocab_size, hidden_layer_neurons1)
    torch.nn.Dropout(0.5)
    self.lin2 = nn.Linear(hidden_layer_neurons1, hidden_layer_neurons2)
    torch.nn.Dropout(0.5)
    if hidden_layer_neurons3 is not None:
      self.lin3 = nn.Linear(hidden_layer_neurons2, hidden_layer_neurons3)
      torch.nn.Dropout(0.5)
      self.lin4 = nn.Linear(hidden_layer_neurons3, num_labels)
    else:
      self.lin3 = nn.Linear(hidden_layer_neurons2, num_labels)
      self.lin4 = None
    
  def forward(self, x):
    out = self.lin1(x)
    out = F.tanh(self.lin2(out))
    out = self.lin3(out)
    if self.lin4 is None:
      return F.softmax(out)
    else:
      return F.softmax(self.lin4(out))
    
class Task5c(nn.Module):
  def __init__(self, num_labels, hidden_layer_neurons1, hidden_layer_neurons2, hidden_layer_neurons3, vocab_size):
    super(Task5c, self).__init__()
    self.lin1 = nn.Linear(vocab_size, hidden_layer_neurons1)
    torch.nn.Dropout(0.5)
    self.lin2 = nn.Linear(hidden_layer_neurons1, hidden_layer_neurons2)
    torch.nn.Dropout(0.5)
    if hidden_layer_neurons3 is not None:
      self.lin3 = nn.Linear(hidden_layer_neurons2, hidden_layer_neurons3)
      torch.nn.Dropout(0.5)
      self.lin4 = nn.Linear(hidden_layer_neurons3, num_labels)
    else:
      self.lin3 = nn.Linear(hidden_layer_neurons2, num_labels)
      self.lin4 = None
    
  def forward(self, x):
    out = self.lin1(x)
    out = F.sigmoid(self.lin2(out))
    out = self.lin3(out)
    if self.lin4 is None:
      return F.softmax(out)
    else:
      return F.softmax(self.lin4(out))

Setting model parameters for Task 5 A

In [0]:
NUM_LABELS = 2  
HIDDEN_LAYER_NEURONS1 = 30
HIDDEN_LAYER_NEURONS2 = 20
HIDDEN_LAYER_NEURONS3 = 10
  
task5a = Task5a(NUM_LABELS, HIDDEN_LAYER_NEURONS1, HIDDEN_LAYER_NEURONS2, HIDDEN_LAYER_NEURONS3, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function5a = nn.NLLLoss()
opt5a = torch.optim.SGD(task5a.parameters(), lr = 0.01)

Setting model parameters for Task 5 B

In [0]:
NUM_LABELS = 2  
HIDDEN_LAYER_NEURONS1 = 30
HIDDEN_LAYER_NEURONS2 = 20
HIDDEN_LAYER_NEURONS3 = 10
  
task5b = Task5b(NUM_LABELS, HIDDEN_LAYER_NEURONS1, HIDDEN_LAYER_NEURONS2, HIDDEN_LAYER_NEURONS3, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function5b = nn.NLLLoss()
opt5b = torch.optim.SGD(task5b.parameters(), lr = 0.01)

Setting model parameters for Task 5 C

In [0]:
NUM_LABELS = 2  
HIDDEN_LAYER_NEURONS1 = 30
HIDDEN_LAYER_NEURONS2 = 20
HIDDEN_LAYER_NEURONS3 = 10
  
task5c = Task5c(NUM_LABELS, HIDDEN_LAYER_NEURONS1, HIDDEN_LAYER_NEURONS2, HIDDEN_LAYER_NEURONS3, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function5c = nn.NLLLoss()
opt5c = torch.optim.SGD(task5c.parameters(), lr = 0.01)

#**Neural network for Task 6**

In [0]:
class Task6(nn.Module):
  def __init__(self, num_labels, hidden_layer_neurons1, hidden_layer_neurons2, hidden_layer_neurons3, vocab_size):
    super(Task6, self).__init__()
    self.lin1 = nn.Linear(vocab_size, hidden_layer_neurons1)
    torch.nn.Dropout(0.5)
    self.lin2 = nn.Linear(hidden_layer_neurons1, hidden_layer_neurons2)
    torch.nn.Dropout(0.5)
    if hidden_layer_neurons3 is not None:
      self.lin3 = nn.Linear(hidden_layer_neurons2, hidden_layer_neurons3)
      torch.nn.Dropout(0.5)
      self.lin4 = nn.Linear(hidden_layer_neurons3, num_labels)
    else:
      self.lin3 = nn.Linear(hidden_layer_neurons2, num_labels)
      self.lin4 = None
    
  def forward(self, x):
    out = self.lin1(x)
    out = F.relu(self.lin2(out))
    out = self.lin3(out)
    if self.lin4 is None:
      return F.softmax(out)
    else:
      return F.softmax(self.lin4(out))

Setting model parameters for Task 6

In [0]:
NUM_LABELS = 2  
HIDDEN_LAYER_NEURONS1 = 30
HIDDEN_LAYER_NEURONS2 = 20
HIDDEN_LAYER_NEURONS3 = 10
  
task6 = Task6(NUM_LABELS, HIDDEN_LAYER_NEURONS1, HIDDEN_LAYER_NEURONS2, HIDDEN_LAYER_NEURONS3, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function6 = nn.NLLLoss()
opt6 = torch.optim.SGD(task6.parameters(), lr = 0.01)

#**Training for Task 6**

**Task 6**

*TRAINING*

In [0]:
task = train(task6, loss_function6, opt6)
calculateMetrics(task)


In [0]:
save_model(task.cpu(), 'model_6.mdl')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Task4. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


#**Neural network for Task 7**

In [0]:
class Task7(nn.Module):
  def __init__(self, num_labels, hidden_layer_neurons1, hidden_layer_neurons2, hidden_layer_neurons3, vocab_size):
    super(Task7, self).__init__()
    self.lin1 = nn.Linear(vocab_size, hidden_layer_neurons1)
    torch.nn.Dropout(0.5)
    self.lin2 = nn.Linear(hidden_layer_neurons1, hidden_layer_neurons2)
    torch.nn.Dropout(0.5)
    if hidden_layer_neurons3 is not None:
      self.lin3 = nn.Linear(hidden_layer_neurons2, hidden_layer_neurons3)
      torch.nn.Dropout(0.5)
      self.lin4 = nn.Linear(hidden_layer_neurons3, num_labels)
    else:
      self.lin3 = nn.Linear(hidden_layer_neurons2, num_labels)
      self.lin4 = None
    
  def forward(self, x):
    out = self.lin1(x)
    out = F.relu(self.lin2(out))
    out = self.lin3(out)
    if self.lin4 is None:
      return F.softmax(out)
    else:
      return F.softmax(self.lin4(out))

Setting model parameters for Task 7a

In [0]:
NUM_LABELS = 2  
HIDDEN_LAYER_NEURONS1 = 30
HIDDEN_LAYER_NEURONS2 = 20
HIDDEN_LAYER_NEURONS3 = 10
  
task7a = Task7(NUM_LABELS, HIDDEN_LAYER_NEURONS1, HIDDEN_LAYER_NEURONS2, HIDDEN_LAYER_NEURONS3, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function7a = nn.CrossEntropyLoss()
opt7a = torch.optim.SGD(task7a.parameters(), lr = 0.01)

Setting model parameters for Task 7b

In [0]:
NUM_LABELS = 2  
HIDDEN_LAYER_NEURONS1 = 30
HIDDEN_LAYER_NEURONS2 = 20
HIDDEN_LAYER_NEURONS3 = 10
  
task7b = Task7(NUM_LABELS, HIDDEN_LAYER_NEURONS1, HIDDEN_LAYER_NEURONS2, HIDDEN_LAYER_NEURONS3, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function7b = nn.MSELoss()
opt7b = torch.optim.SGD(task7b.parameters(), lr = 0.01)

Setting model parameters for Task 7c

In [0]:
NUM_LABELS = 2  
HIDDEN_LAYER_NEURONS1 = 30
HIDDEN_LAYER_NEURONS2 = 20
HIDDEN_LAYER_NEURONS3 = 10
  
task7c = Task7(NUM_LABELS, HIDDEN_LAYER_NEURONS1, HIDDEN_LAYER_NEURONS2, HIDDEN_LAYER_NEURONS3, VOCAB_SIZE).cuda()
# define a loss function and an optimizer
loss_function7c = nn.HingeEmbeddingLoss()
opt7c = torch.optim.SGD(task7c.parameters(), lr = 0.20)

#**Training for Task 7**

**Task 7 a**

*TRAINING*

In [25]:
task = train(task7a, loss_function7a, opt7a)
calculateMetrics(task)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Time Taken - 25.428458213806152
Epoch - 0 , LOSS - tensor(0.3133, device='cuda:0')
Time Taken - 25.342161655426025
Epoch - 1 , LOSS - tensor(0.3136, device='cuda:0')
Time Taken - 25.43352437019348
Epoch - 2 , LOSS - tensor(0.3133, device='cuda:0')
Time Taken - 25.365437984466553
Epoch - 3 , LOSS - tensor(0.3133, device='cuda:0')
Time Taken - 25.402482986450195
Epoch - 4 , LOSS - tensor(0.3133, device='cuda:0')
Time Taken - 25.489394187927246
Epoch - 5 , LOSS - tensor(0.3133, device='cuda:0')
Time Taken - 25.39272165298462
Epoch - 6 , LOSS - tensor(0.3133, device='cuda:0')
Time Taken - 25.402258157730103
Epoch - 7 , LOSS - tensor(0.3133, device='cuda:0')
Time Taken - 25.493067741394043
Epoch - 8 , LOSS - tensor(0.3133, device='cuda:0')
Time Taken - 25.804034948349
Epoch - 9 , LOSS - tensor(0.3133, device='cuda:0')
--- AFTER TRAINING ---
Accuracy - 0.8652
Recall - 0.8856235107227959
Precision - 0.8524464831804281
Fmeasure - 0.8687183482664589


In [26]:
save_model(task.cpu(), 'model_7a.mdl')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Task7. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


**Task 7 b**

*TRAINING*

In [0]:
task = train(task7b, loss_function7b, opt7b)
calculateMetrics(task)


In [0]:
save_model(task.cpu(), 'model_7b.mdl')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Task4. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


**Task 7 c**

*TRAINING*

In [39]:
task = train(task7c, loss_function7c, opt7c)
calculateMetrics(task)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Time Taken - 33.00462055206299
Epoch - 0 , LOSS - tensor(1., device='cuda:0')
Time Taken - 32.24803566932678
Epoch - 1 , LOSS - tensor(1., device='cuda:0')
Time Taken - 32.04449462890625
Epoch - 2 , LOSS - tensor(1., device='cuda:0')
Time Taken - 32.094396352767944
Epoch - 3 , LOSS - tensor(1., device='cuda:0')
Time Taken - 32.16407775878906
Epoch - 4 , LOSS - tensor(1., device='cuda:0')
Time Taken - 31.97115182876587
Epoch - 5 , LOSS - tensor(1., device='cuda:0')
Time Taken - 32.02061724662781
Epoch - 6 , LOSS - tensor(1., device='cuda:0')
Time Taken - 32.06936430931091
Epoch - 7 , LOSS - tensor(1., device='cuda:0')
Time Taken - 32.247437715530396
Epoch - 8 , LOSS - tensor(1., device='cuda:0')
Time Taken - 31.875512838363647
Epoch - 9 , LOSS - tensor(1., device='cuda:0')
--- AFTER TRAINING ---
Accuracy - 0.5036
Recall - 1.0
Precision - 0.5036
Fmeasure - 0.6698590050545358


In [0]:
save_model(task.cpu(), 'model_7c.mdl')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Task4. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [0]:
from google.colab import files
temp_test = files.upload()


In [0]:
import torch
bow = torch.load(io.BytesIO(temp_test['model.json']))

In [0]:
from google.colab import files
temp_test = files.upload()

In [0]:
pickle_in = open("word_to_ix.pickle","rb")
word_to_ix = pickle.load(pickle_in)